# Загрузка Pandas и очистка данных

In [21]:
import pandas as pd
from ast import literal_eval
import numpy as np
import re

In [22]:
df = pd.read_csv('main_task_new.csv')

In [23]:
# Создаем функцию для заполнения dummie-атрибутов стилей кухни
def fill_dummie_cuisine_style(row):
    for cuisine in row['Cuisine Style']:
        row[cuisine] = 1
    return row


# Функция разбиения дат на 2 столбца
def date_split(row):
    if len(row['Reviews_date']) == 2:
        return row['Reviews_date'][0], row['Reviews_date'][1]
    elif len(row['Reviews_date']) == 1:
        return row['Reviews_date'][0], np.nan
    else:
        return np.nan,np.nan

In [24]:
# Заполняем пропуски 'Price Range' значением '$$ - $$$'. Создаем dummie-атрибуты из 'Price Range'
df['Price Range'].fillna('$$ - $$$', inplace=True)
df = pd.concat([df, pd.get_dummies(df['Price Range'])], axis=1)

# Создаем dummie-атрибуты из 'City'
df = pd.concat([df, pd.get_dummies(df.City, dummy_na=True)], axis=1)

# Заполняем пропуски стилей кухонь 'Other' и преобразуем строки в списки 
df['Cuisine Style'].fillna("['Other']", inplace=True)
df['Cuisine Style'] = df['Cuisine Style'].apply(literal_eval)

# Создаем новый атрибут "количество стилей кухни"
df['Cuisine_Style_count'] = df['Cuisine Style'].apply(len)

# Создаем новый признак, в котором отмечаем строки, в которых были пропуски в столбце 'Number of Reviews'
df['Number_of_Reviews_isNAN'] = pd.isna(df['Number of Reviews']).astype('uint8')

# Заполняем пропуски 'Number of Reviews' средним значением
df['Number of Reviews'].fillna(np.mean(df['Number of Reviews']), inplace=True)

# Заполняем dummie-атрибуты стилей кухни единицами
df = df.apply(fill_dummie_cuisine_style, axis=1)

# Создаем шаблон для поиска дат в тексте отзывов
pattern = re.compile('\'\d+\/\d+\/\d+\'?')  
# выводим даты в отзывах в отдельный столбец
df['Reviews_date'] = df['Reviews'].apply(pattern.findall)
# трансформируем содержание столбца в формат datetime
df['Reviews_date'] = df['Reviews_date'].apply(lambda x: sorted([pd.to_datetime(i).date() for i in x]))
# Создаем новые признаки, которые заполняем значениями дат
df[['date_1', 'date_2']] = df[['Reviews_date']].apply(date_split, axis=1, result_type='expand')
# Заколняем пропуски в 'date_1', 'date_2' на '0'
df[['date_1', 'date_2']].fillna(0,inplace=True)
# Вычисляем разницу между датами
df['date_dif'] = df['date_2'] - df['date_1']
# Приводим разницу между датами в формат чисел
df['date_dif'] = pd.to_numeric(df['date_dif'])

# Удаляем лишние столбцы
df = df.drop(['Restaurant_id', 'City', 'Cuisine Style', 'Price Range', 'Reviews', 'URL_TA', 'ID_TA', 
              'Reviews_date', 'date_1', 'date_2'], axis=1)

# Заполняем пропуски нулями
df = df.fillna(0)

C:\Users\gromf\anaconda3\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [25]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [11]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [26]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [13]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [27]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [28]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.21524649999999998
